Construyendo tu Agente de IA: Integrando LangChain y Amazon Bedrock para consultas de datos privados

Librerias necesarias

In [ ]:
!pip install langchain_community langchain_core langchain pypdf boto3 langchain_aws

Credenciales para usar AWS Bedrock

In [ ]:
import getpass
import os
os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter your AWS_ACCESS_KEY_ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("Enter your AWS_SECRET_ACCESS_KEY: ")
os.environ["AWS_DEFAULT_REGION"] = getpass.getpass("Enter your AWS_DEFAULT_REGION: ")

In [ ]:
import boto3

"""
Configura la sesión de AWS
"""
aws_access_key = os.environ.get("AWS_ACCESS_KEY_ID")
aws_secret_key = os.environ.get("AWS_SECRET_ACCESS_KEY")
region = os.environ.get("AWS_DEFAULT_REGION")

session = boto3.Session(
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name=region
)

Cargar documento PDF

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("documento.pdf")
docs = loader.load()

In [ ]:
Text Splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

documents = text_splitter.split_documents(docs)

In [ ]:
Crear modelo de embedding

In [ ]:
from langchain.embeddings import BedrockEmbeddings

"""Crea una instancia del modelo de embeddings de Bedrock"""
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=os.environ['AWS_DEFAULT_REGION']
)

# Crear el modelo de embeddings usando el modelo Titan de Amazon
embeddings = BedrockEmbeddings(
    client=bedrock_client,
    model_id="amazon.titan-embed-text-v1"  # Modelo de embeddings de Amazon
)

Base de datos vectorial

In [ ]:
from langchain.vectorstores import Chroma

"""Crea un almacén vectorial Chroma usando embeddings de Bedrock"""
# Obtener el modelo de embeddings
embedding_model = embeddings

NOMBRE_INDICE_CHROMA = "embeddings-documents"

# Crear el almacén vectorial
vectorstore_chroma = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=NOMBRE_INDICE_CHROMA
)

#Hacer que nuestra vectorstore persista en nuestro disco
vectorstore_chroma.persist()

vectorstore = Chroma(
    persist_directory=NOMBRE_INDICE_CHROMA,
    embedding_function=embedding_model
)

Retriever

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)

In [ ]:
query = "consulta"

retriever.invoke(query)

Convertir al retriever a herramienta

In [ ]:
from langchain.tools.retriever import create_retriever_tool

txt = '''
  Utiliza esta herramienta para buscar información reacionada
  con las preguntas que haga el usuario
'''
retriever_tool = create_retriever_tool(
    retriever,
    "RetrieverTool",
    txt
)

Declaramos las Tools que usara el agente

In [ ]:
tools = [retriever_tool]

LLM Bedrock

In [ ]:
import boto3
from langchain_aws.llms.bedrock import BedrockLLM

llm = BedrockLLM(
    client=bedrock_client,
    model_id="anthropic.claude-instant-v1",
    model_kwargs={
        "max_tokens_to_sample": 300,
        "temperature": 0.1,
        "top_p": 0.9,
    }
)

PromptTemplate

In [ ]:
from langchain.prompts import PromptTemplate

# Template para el agente
prompt = PromptTemplate.from_template("""
  Eres un asistente útil con acceso a las siguientes herramientas:

  {tools}

  Utilice el siguiente formato:

  Pregunta: la pregunta de entrada que debes responder
  Pensamiento: siempre debes pensar en qué hacer
  Acción: la acción a realizar debe ser una de [{tool_names}]
  Entrada de acción: la entrada a la acción.
  Observación: el resultado de la acción.
  ... (este Pensamiento/Acción/Introducción de Acción/Observación puede repetirse N veces)
  Pensamiento: ahora sé la respuesta final
  Respuesta final: la respuesta final a la pregunta de entrada original

  Pregunta:{input}

  {agent_scratchpad}
  """)

Agente

In [ ]:
from langchain.agents import Tool, AgentExecutor, create_react_agent

# Create the React agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

# Crear el ejecutor del agente
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

Ejemplo de uso del agente

In [ ]:
result = agent_executor.invoke({
        "input": "Consulta"
})

result